In [ ]:
#try this with arcpy and pandas and see if it's possible
#might just be some spatial joins to contend with and then use pandas for everything else

#import arcpy
from arcpy import env
import geopandas as gpd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
import pandas as pd
#import zipfile
from urllib.request import urlopen
import rasterio

import sys
sys.path.append("..")

from src.features.custom_functions import *

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
#input datasets
town_name = 'Revere'

datasets_dir = r'K:\DataServices\datasets'


gdb = r'K:\DataServices\Projects\Current_Projects\Climate_Change\MVP_MMC_CoolRoofs_MVP\ArcGIS\CoolRoofs_Analysis.gdb'
env.overwriteOutput = True
#arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 StatePlane Massachusetts FIPS 2001 (Meters)")
rooftops_layer_name = town_name + '_footprints_with_info'
rooftops_layer = gpd.read_file(gdb, layer=rooftops_layer_name)



In [79]:
#get shapefile from a massgis link
shapefile_excel = 'https://www.mass.gov/doc/massgis-parcel-data-download-links-table/download'
#shapefile_lookup = pd.read_excel(shapefile_excel)
#Actually maybe try that later, for now we can repurpose the 3A scripts and use data from the K drive



In [106]:
town_structures_w_info = cool_roof_parcel_process(town_name=town_name, structures_data=rooftops_layer)